##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

<table class="tfa-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders-addons/blob/master/docs/tutorials/dynamic_embedding_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview
In this tutorial, we're going to use the rating data to predict the user's rating of other movies. To achieve this goal, we will follow the following steps:

1. Get our data and do some preprocessing to get the required format.
2. Implement a neural collaborative filtering(NeuralCF) model.
3. Train the model.

Different from the general recommendation model, the model we implemented replaces `tf.nn.embedding_lookup` with `tfra.dynamic_embedding.embedding_lookup`, which can handle super large sparse features.

## Imports
Let's first get our imports out of the way.

In [1]:
# !pip install -q --upgrade tensorflow-recommenders-addons
# !pip install -q --upgrade tensorflow-datasets

In [1]:
import sys,os

In [2]:
os.environ['TF_GPU_THREAD_MODE']='gpu_private'

In [3]:
sys.version

'3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]'

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [5]:
tf.version.VERSION

'2.8.0'

In [6]:
# make TF not take all the GPU mem
gpus = tf.config.list_physical_devices('GPU')

# Single GPU
# gpu = gpus[1]
# tf.config.set_visible_devices(gpu, 'GPU')
# tf.config.experimental.set_memory_growth(gpu, True)

# # Multi GPUs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(f'gpus: {gpus}')

gpus: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [7]:
import tensorflow_datasets as tfds
import tensorflow_recommenders_addons as tfra

In [8]:
tfra.__version__

'0.3.3-dev'

In [9]:
from IPython.core.debugger import set_trace

## Preparing the dataset

This tutorial uses movies reviews provided by the MovieLens 100K dataset, a classic dataset from the GroupLens research group at the University of Minnesota. In order to facilitate processing, we convert the data type of `movie_id` and `user_id` into `int64`.


In [10]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_id": tf.strings.to_number(x["movie_id"], tf.int64),
    "user_id": tf.strings.to_number(x["user_id"], tf.int64),
    "user_rating": x["user_rating"]
})

tf.random.set_seed(2021)
shuffled = ratings.shuffle(100_000, seed=2021, reshuffle_each_iteration=False)

dataset_train = shuffled.take(100_000).batch(256)

## Implementing a model
The NCFModel we implemented is very similar to the conventional one, and the main difference lies in the embedding layer. We specify the variable of embedding layer by `tfra.dynamic_embedding.get_variable`. 

In [11]:
class NCFModel(tf.keras.Model):

    def __init__(self):
        super(NCFModel, self).__init__()
        self.embedding_size = 32
        self.d0 = Dense(
            256,
            activation='relu',
            kernel_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1),
            bias_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1))
        self.d1 = Dense(
            64,
            activation='relu',
            kernel_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1),
            bias_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1))
        self.d2 = Dense(
            1,
            kernel_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1),
            bias_initializer=tf.keras.initializers.RandomNormal(0.0, 0.1))
        
        self.user_embeddings = tfra.dynamic_embedding.get_variable(
            name="user_dynamic_embeddings",
            dim=self.embedding_size,
            initializer=tf.keras.initializers.RandomNormal(-1, 1))
        self.user_embeddings_shadow = tfra.dynamic_embedding.shadow_ops.ShadowVariable(
            self.user_embeddings,
            name='user_dynamic_embeddings_shadow',
            max_norm=None,
            trainable=True)
        
        self.movie_embeddings = tfra.dynamic_embedding.get_variable(
            name="moive_dynamic_embeddings",
            dim=self.embedding_size,
            initializer=tf.keras.initializers.RandomNormal(-1, 1))
        self.movie_embeddings_shadow = tfra.dynamic_embedding.shadow_ops.ShadowVariable(
            self.movie_embeddings,
            name='movie_dynamic_embeddings_shadow',
            max_norm=None,
            trainable=True)
        
        self.loss = tf.keras.losses.MeanSquaredError()

    def call(self, batch):
        movie_id = batch["movie_id"]
        second_movie_id = tf.stack([tf.random.shuffle(batch["movie_id"]), tf.random.shuffle(batch["movie_id"])], axis=1)
        user_id = batch["user_id"]
        rating = batch["user_rating"]

        input_shape = tf.shape(user_id)
        user_id_weights = tfra.dynamic_embedding.shadow_ops.embedding_lookup(self.user_embeddings_shadow, user_id)
        user_id_weights = tf.reshape(user_id_weights, tf.concat([input_shape, [self.embedding_size]], 0))
        
        tf.print(user_id_weights)
        input_shape = tf.shape(movie_id)
        movie_id_weights = tfra.dynamic_embedding.shadow_ops.embedding_lookup(self.movie_embeddings_shadow, movie_id)
        movie_id_weights = tf.reshape(movie_id_weights, tf.concat([input_shape, [self.embedding_size]], 0))
        
        input_shape = tf.shape(second_movie_id)
        second_movie_id_weights = tfra.dynamic_embedding.shadow_ops.embedding_lookup(self.movie_embeddings_shadow, second_movie_id)
        second_movie_id_weights = tf.reshape(second_movie_id_weights, tf.concat([input_shape, [self.embedding_size]], 0))
        second_movie_id_weights = tfra.dynamic_embedding.keras.layers.embedding.reduce_pooling(second_movie_id_weights)
        
        embeddings = tf.concat([user_id_weights, movie_id_weights, second_movie_id_weights], axis=1)

        dnn = self.d0(embeddings)
        dnn = self.d1(dnn)
        dnn = self.d2(dnn)
        out = tf.reshape(dnn, shape=[-1])
        loss = self.loss(rating, out)
        return loss

Let's instantiate the model, and wrap the optimizer in tfra.dynamic_embedding.DynamicEmbeddingOptimizer.

In [12]:
model = NCFModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tfra.dynamic_embedding.DynamicEmbeddingOptimizer(optimizer)

## Training the model
After defining the model, we can train the model and observe the change of loss.

In [13]:
# @tf.function
def train_step(batch, model):
    with tf.GradientTape() as tape:
        loss = model(batch)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [14]:
def train(epoch=1):
    for i in range(epoch):
        total_loss = np.array([])
        for (_, batch) in enumerate(dataset_train):
            loss = train_step(batch, model)
            total_loss = np.append(total_loss, loss)
        print("epoch:", i, "mean_squared_error:", np.mean(total_loss))

In [15]:
train(1)

[[-1.54358864 -0.184731364 -2.0934484 ... -0.697834492 -1.60195971 -2.53917217]
 [-2.0909338 -0.0631753206 0.357673407 ... -1.14787972 -0.431725979 -1.40734792]
 [1.46237206 -1.48622155 -1.45492768 ... 0.693823814 0.0290193558 -0.166430533]
 ...
 [-0.388025165 -0.34725374 -1.80389428 ... -1.40494144 -1.46805191 -1.12663031]
 [-0.381850302 0.53756249 -0.58995831 ... -2.15224719 -1.10494184 -1.73442006]
 [-1.4699868 0.584298253 -0.219314933 ... -0.397144377 0.53237164 0.870266318]]


InvalidArgumentError: Inputs to operation AddN of type AddN must have the same size and shape.  Input 0: [256,32] != input 1: [256,2,32] [Op:AddN]